In [11]:
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, classification_report, f1_score, recall_score
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, confusion_matrix, classification_report, f1_score, recall_score
from sklearn.cluster import KMeans
from joblib import dump, load

Import del dataset in un DataFrame

In [3]:
data = pd.read_csv('Data/25day_dataset.csv')

Si etichetta il dataset con 4 classi:

    1) Tutti i dispositivi spenti [0]
    2) Solo la lavatrice accesa [1]
    3) Solo la lavastoviglie accesa [2]
    4) Solo il forno acceso [3]
    
Le altre combinazioni di dispositivi accesi non si verificano mai e quindi vengono esclusi a priori. 

In [4]:
label = []
for i in range(len(data)):
    if data['wahing_machine'].iloc[i]==0 and data['dishwasher'].iloc[i]==0 and data['oven'].iloc[i]==0:
        label.append(0)
    elif data['wahing_machine'].iloc[i]>0:
        label.append(1)
    elif data['dishwasher'].iloc[i]>0:
        label.append(2)
    elif data['oven'].iloc[i]>0:
        label.append(3)
data['Class'] = label

Codice per la creazione delle fold. Di seguito le istruzioni fronite per effettuare il corretto folding:

Test set fold 01: Tutte le misure del giorno 2022-01-01

Test set fold 02: Tutte le misure del giorno 2022-01-02

Test set fold 03: Tutte le misure del giorno 2022-01-03

…..

Test set fold 09: Tutte le misure del giorno 2022-01-10 Tutte le misure del giorno 2022-01-11

Test set fold 10: Tutte le misure del giorno 2022-01-12 Tutte le misure del giorno 2022-01-13 (dati disponibili fino alle 03:16)

In [5]:
Test_set1 =data[data['DateTime'].str.startswith('2022-01-01')]
Test_set2 =data[data['DateTime'].str.startswith('2022-01-02')]
Test_set3 =data[data['DateTime'].str.startswith('2022-01-03')]
Test_set4 =data[data['DateTime'].str.startswith('2022-01-04')]
Test_set5 =data[data['DateTime'].str.startswith('2022-01-05')]
Test_set6 =data[data['DateTime'].str.startswith('2022-01-06')]
Test_set7 =data[data['DateTime'].str.startswith('2022-01-07')]
Test_set8 =data[(data['DateTime'].str.startswith('2022-01-08'))]
Test_set9 =data[(data['DateTime'].str.startswith('2022-01-10')) | (data['DateTime'].str.startswith('2022-01-11'))]
Test_set10 =data[(data['DateTime'].str.startswith('2022-01-12')) | (data['DateTime'].str.startswith('2022-01-13'))]
test_set_list= [Test_set1, Test_set2, Test_set3, Test_set4, Test_set5, Test_set6, Test_set7, Test_set8, Test_set9, Test_set10]

Train e test del modello per ogni fold creata nel passo precedente

In [14]:
#Dizionari per salvarsi le prestazioni richieste 
f_score={}
recall={}

#Ciclo per scorrere le fold di test e trainare un modello per ognuna di esse
-for i in range(len(test_set_list)):
    
    #Creazione delle X_train e y_train togliendo tutti i record del test set, cioè la fold i-esima
    dfi = data.drop(test_set_list[i].index)
    y_= dfi.Class
    X_= dfi.drop(['DateTime', 'Class', 'wahing_machine', 'dishwasher', 'oven'], axis=1)
    
    #Prendo intervalli di 5 valori consecutivi e li sostituisco con la media...
    X = X_.groupby(np.arange(len(dfi))//5).mean()
    #...per le classi scelgo quella piu rappresentata dei 5 campioni
    y = y_.groupby(np.arange(len(dfi))//5).agg(lambda x:x.value_counts().index[0])

    #Undersampling per equilibrare il dataset altrimenti molto sbilanciato
    two= int((y.values == 2).sum()) 
    three= int((y.values == 3).sum())
    undersample = RandomUnderSampler(sampling_strategy={0: 10000, 1: 10000, 2: two, 3: three})
    X_new, y_new = undersample.fit_resample(X, y)
    
    #Inizializzazione del modello della Random Forest con i parametri scelti tramite la validation
    clf = RandomForestClassifier(
                                   class_weight= {0: 9, 1: 1, 2: 1, 3: 6},
                                   criterion= 'entropy', 
                                   max_depth= 10,
                                   max_features= 'sqrt',
                                   max_samples= 0.3,
                                   n_estimators= 300
                                )
    
    #fit del modello e salvo su file  
    clf.fit(X_new, y_new)
    dump(clf, f'Models_RF/RandomForest_Fold{i}.sav')
    
    #Creazione di X_test e y_test prendendo i campioni della fold 
    X_test = test_set_list[i].copy().drop(['DateTime', 'Class', 'wahing_machine', 'dishwasher', 'oven'], axis=1)
    y_test = test_set_list[i].Class
    
    #Predizione del modello sulle X_test
    y_preds = clf.predict(X_test)
    
    #Stampa della matrice di confusione generata dalla predizione e relative performance ottenute
    print("CM:\n" + str(confusion_matrix(y_test,y_preds)) + "\n")
    print(classification_report(y_test,y_preds))
    
    #Calcolo delle metriche di F1_Score e Recall per ogni classe
    r = recall_score(y_test,y_preds, average= None) 
    f = f1_score(y_test,y_preds, average= None)
    f_score[i] = f
    recall[i] = r 
 
#Calcolo delle metriche medie ottenute da tutte le fold
f1= pd.DataFrame.from_dict(f_score, orient='index').replace(0, np.nan)
print(f1.mean(axis=0, skipna=True))
r1= pd.DataFrame.from_dict(recall, orient='index').replace(0, np.nan)
print(r1.mean(axis=0, skipna=True))

CM:
[[80403   494   720    48]
 [   78  3975     0     0]
 [    0     0     0     0]
 [  143     0     0   539]]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     81665
           1       0.89      0.98      0.93      4053
           2       0.00      0.00      0.00         0
           3       0.92      0.79      0.85       682

    accuracy                           0.98     86400
   macro avg       0.70      0.69      0.69     86400
weighted avg       0.99      0.98      0.99     86400



C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

CM:
[[76726   841   267   435]
 [  634  6371     0     3]
 [    0     0     0     0]
 [   53     0     0  1070]]

              precision    recall  f1-score   support

           0       0.99      0.98      0.99     78269
           1       0.88      0.91      0.90      7008
           2       0.00      0.00      0.00         0
           3       0.71      0.95      0.81      1123

    accuracy                           0.97     86400
   macro avg       0.65      0.71      0.67     86400
weighted avg       0.98      0.97      0.98     86400



C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

CM:
[[78985   807   274  1029]
 [  105   873     0    46]
 [  419   128  3030     0]
 [    0     0     0   704]]

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     81095
           1       0.48      0.85      0.62      1024
           2       0.92      0.85      0.88      3577
           3       0.40      1.00      0.57       704

    accuracy                           0.97     86400
   macro avg       0.70      0.92      0.76     86400
weighted avg       0.98      0.97      0.97     86400

CM:
[[81536    62    32   125]
 [    0     0     0     0]
 [    0     0     0     0]
 [  509  1226     4  2906]]

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     81755
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.96      0.63      0.76      4645

    accuracy                           0.98     86400
   macro av

...

CM:
[[78952   383  1729   375]
 [  128  4027     0     0]
 [    0     0     0     0]
 [  273     3     0   530]]

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     81439
           1       0.91      0.97      0.94      4155
           2       0.00      0.00      0.00         0
           3       0.59      0.66      0.62       806

    accuracy                           0.97     86400
   macro avg       0.62      0.65      0.64     86400
weighted avg       0.99      0.97      0.98     86400



C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

CM:
[[166634    419    542    636]
 [   111   2673      0      6]
 [     0      0      0      0]
 [   663      0      0   1116]]

              precision    recall  f1-score   support

           0       1.00      0.99      0.99    168231
           1       0.86      0.96      0.91      2790
           2       0.00      0.00      0.00         0
           3       0.63      0.63      0.63      1779

    accuracy                           0.99    172800
   macro avg       0.62      0.64      0.63    172800
weighted avg       0.99      0.99      0.99    172800



C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\massi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1

CM:
[[157684    815    712    711]
 [   374   6112      0      4]
 [   232      1   3230      0]
 [    81      1      1   2842]]

              precision    recall  f1-score   support

           0       1.00      0.99      0.99    159922
           1       0.88      0.94      0.91      6490
           2       0.82      0.93      0.87      3463
           3       0.80      0.97      0.88      2925

    accuracy                           0.98    172800
   macro avg       0.87      0.96      0.91    172800
weighted avg       0.98      0.98      0.98    172800

0    0.989574
1    0.874846
2    0.891608
3    0.727138
dtype: float64
0    0.984339
1    0.937444
2    0.883947
3    0.793448
dtype: float64


In [40]:
r1.iloc[:, 1:].mean(axis=0, skipna=True).mean()

0.8716128670992677

In [39]:
print(f1.transpose().iloc[3])

0    0.849488
1    0.813379
2    0.567056
3    0.757165
4         NaN
5         NaN
6    0.702558
7    0.619521
8    0.631043
9    0.876890
Name: 3, dtype: float64
